In [2]:
# Split up the dataset randomly into 80% train set, 10% dev set, 10% test set.
# Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
import random # shuffle the list of words to get an even distribution
import torch
import torch.nn.functional as F

words = open('../names.txt', 'r').read().splitlines()
random.seed(230)
random.shuffle(words)


In [3]:
# Map the words to indexes
chars = sorted(list(set(''.join(words)))) # get the unique characters through the set() method
stoi = {s:i +1 for i,s in enumerate(chars)} # string to index
stoi['.'] = 0 # end character

itos = {i:s for s,i in stoi.items()} # index to string
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [4]:
# BUILD THE DATASET
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  xs, ys = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      xs.append(context)
      ys.append(ix)
      context = context[1:] + [ix] # crop and append

  xs = torch.tensor(xs)
  ys = torch.tensor(ys)
  return xs, ys

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

In [5]:
# BOILERPLATE CODE DONE

# Utility function that compares our manually calculated gradients to pytorch (assumed source of truth) to check accuracy
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
## SET UP MODEL PARAMETERS
gain = 5/3 # we need a gain because we are using the tanh activation function; squashes, add a gain to get back to normal std
dim_emb = 10 # dimensionality of the embedding
n_hidden = 64 # number of hidden units

# set up the model drivers
g = torch.Generator().manual_seed(2147483647)
emb_lookup = torch.randn((len(chars) + 1, dim_emb), generator=g) #also written as 'C'. Can scale up dimensionality to capture more nuanced patterns

# layer one
W1 = torch.randn((dim_emb * block_size, n_hidden), generator=g)  * (gain / (dim_emb * block_size) **0.5) 
b1 = torch.randn(n_hidden, generator=g) * 0.01

# layer two
W2 = torch.randn((n_hidden, len(chars) + 1), generator=g) * (gain / n_hidden**0.5) # ending back up with 27 possible outputs / chars
b2 = torch.randn(len(chars) + 1, generator=g) * 0

# batchnorm parameters
bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0 # scale
bnbias = torch.zeros((1, n_hidden)) * 0.1 # shift

# put all of the parameters in one array for neatness -- you can sum all these to get total param count
parameters = [emb_lookup, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [7]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience

# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = emb_lookup[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation

# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer

# Linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb)), but implementing manually again
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean() # this indexing expression selects one element from each row of logprobs using a pair of arrays (range(n) and Yb) for indexing. For each example i, it picks out logprobs[i, Yb[i]], which is the log probability of the true class Yb[i] for the i-th example. This creates a 1D array where each element corresponds to the log probability assigned by the model to the true class of each example.

# PyTorch backward pass so we can run comparisons against it
for p in parameters:
  p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.7302, grad_fn=<NegBackward0>)

In [216]:
print(emb_lookup.shape)
print(emb_lookup)

# basically, 27 characters, with ten different dimensions

torch.Size([27, 10])
tensor([[ 1.5674e+00, -2.3729e-01, -2.7385e-02, -1.1008e+00,  2.8588e-01,
         -2.9644e-02, -1.5471e+00,  6.0489e-01,  7.9136e-02,  9.0462e-01],
        [-4.7125e-01,  7.8682e-01, -3.2844e-01, -4.3297e-01,  1.3729e+00,
          2.9334e+00,  1.5618e+00, -1.6261e+00,  6.7716e-01, -8.4040e-01],
        [ 9.8488e-01, -1.4837e-01, -1.4795e+00,  4.4830e-01, -7.0731e-02,
          2.4968e+00,  2.4448e+00, -6.7006e-01, -1.2199e+00,  3.0314e-01],
        [-1.0725e+00,  7.2762e-01,  5.1114e-02,  1.3095e+00, -8.0220e-01,
         -8.5042e-01, -1.8068e+00,  1.2523e+00, -1.2256e+00,  1.2165e+00],
        [-9.6478e-01, -2.3211e-01, -3.4762e-01,  3.3244e-01, -1.3263e+00,
          1.1224e+00,  5.9641e-01,  4.5846e-01,  5.4011e-02, -1.7400e+00],
        [ 1.1560e-01,  8.0319e-01,  5.4108e-01, -1.1646e+00,  1.4756e-01,
         -1.0006e+00,  3.8012e-01,  4.7328e-01, -9.1027e-01, -7.8305e-01],
        [ 1.3506e-01, -2.1161e-01, -1.0406e+00, -1.5367e+00,  9.3743e-01,
         -8

In [215]:
print(Xb.shape)
print(Xb)
# 32 examples, 3 characters in each minibatch, where each integer represents the index of the character in the stoi dictionary

torch.Size([32, 3])
tensor([[12, 15, 26],
        [ 1, 19, 25],
        [ 0,  0,  3],
        [ 0,  0,  0],
        [ 1, 12,  4],
        [ 0,  0,  0],
        [12,  5, 18],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [14,  1, 13],
        [ 1,  4,  1],
        [ 0, 11,  5],
        [ 2,  5, 25],
        [ 0, 11,  1],
        [12, 25, 12],
        [ 0, 26, 19],
        [11,  1, 18],
        [ 5,  1, 12],
        [ 1,  4, 25],
        [ 1, 20,  8],
        [ 0,  0,  0],
        [12, 13,  1],
        [ 0,  0,  3],
        [19, 21, 18],
        [ 8,  5, 15],
        [15, 18, 18],
        [ 2,  9,  7],
        [ 0,  0,  5],
        [18,  1,  6],
        [ 0,  0, 23],
        [22,  9, 15],
        [13,  1, 11]])


In [228]:
emb_lookup[12]

# this is the final lookup table that contains the row vectors for each character in the minibatch

tensor([-0.5614, -0.1375, -0.1380, -2.0977, -0.7924,  0.6069, -1.4777, -0.5103,
         0.5642,  0.9684], grad_fn=<SelectBackward0>)

In [235]:
emb_lookup[Xb[:1]]

# go through row by row, concenate the emb_lookup per indicated indices

tensor([[[-0.5614, -0.1375, -0.1380, -2.0977, -0.7924,  0.6069, -1.4777,
          -0.5103,  0.5642,  0.9684],
         [ 0.5557,  0.4746, -1.3867,  1.6229,  0.1720,  0.9885,  0.5066,
           1.0198, -1.9062, -0.4275],
         [-0.9465, -0.1594, -0.1934, -0.3766, -0.0492,  0.0939, -0.6453,
           1.2108, -0.7820,  0.3845]]], grad_fn=<IndexBackward0>)

In [15]:
Yb # when getting the loss, you're going to Yb, plucking out each row (only 1 in this case), then taking the log probabilities for each item, and then taking the mean of that

tensor([ 1,  1,  8,  7, 15,  1,  0, 16, 10,  0, 12,  5, 12, 23,  1, 15, 13,  0,
        14,  1,  9,  8,  1,  9, 18, 25, 25, 14,  1,  5, 14,  9])

Roughly, this is loss:\
loss = -logprobs[range(n), Yx].mean()\
= -(probA + probB + probC) / 3\
= (-1/3)probA + (-1/3)probB + (-1/3)probC\
Thus, dloss / dproba = -1/3, assuming A is the true class because the other items are false classes and don't contribute to the loss. We assign a gradient to A so that the model will update the probability of A next time.

This generalizes to dloss / dlogprobs = (-1/n) for all n True class items.\
Let's say that logprobs = ((-1)probA + (-1)probB + (-1)probC ... + (-1)probN) \
You're looking for the correct label and the gradient changes based on that, so loss is actually the negative log probability of the correct label. So you end up with:\
-1 / n, because all the other ones get derived out.

Note that when we say dlogprobs, this is dloss / dlogprobs.

In [257]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs) # empty gradients for the log probabilities
dlogprobs[range(n), Yb] = -1/n # add gradient to all true classes
cmp('logprobs', dlogprobs, logprobs)

# chain rule -> dprobs wrt loss = dprobs wrt logprobs * dlogprobs wrt loss
dprobs = (1.0 / probs) # dlogprobs / dprobs
dprobs = dprobs * dlogprobs
cmp('probs', dprobs, probs)

# probs = counts * counts_sum_inv, e.g. with shapes [32, 27] * [32, 1] 
# this means that there are two operations: 1) replicating columns in counts_sum_inv, and 2) multiplying
# we first backpropagate through the multiplication then through the replication.
# Recall in micrograd if multiple inputs arrive at a node, you sum. 
# the columns of counts_sum_inv are used multiple times (32) in each column in counts so we'll sum horizontally each column in counts
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True) # [32, 27], then summing all columns (item 1)
# notice also that some gradient is flowing from csi -> probs, so we need to chain through those as well
# dL / dcounts = dprobs / dcounts * dL / dprobs. dprobs / dcounts = counts_sum_inv(counts**0)
dcounts = dprobs * counts_sum_inv
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# counts_sum_inv = counts_sum^-1
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

# counts_sum = counts.sum(1, keepdims=True)
# backproprogate through the sum operation
dcounts += torch.ones_like(counts) * dcounts_sum # multiplying each row (torch.ones just to get the right shape), each element in a given row gets same gradient
cmp('counts', dcounts, counts)

# counts = norm_logits.exp() eg. e^x for every element in norm_logits
# for every element in norm_logits, f(x) = e^x. f'(x) = e^x, still.
# so with the chain rule, dL / dnorm_logits = dL / dcounts * dcounts / dnorm_logits
dnorm_logits = dcounts * norm_logits.exp()
cmp('norm_logits', dnorm_logits, norm_logits)

# norm_logits = logits - logit_maxes
# dL / dlogit_maxes = dnorm_logits / dlogit_maxes * dL / dnorm_logits
# logit_maxes is being replicated again, also
dlogit_maxes = -dnorm_logits.sum(1, keepdims=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
# dL / dlogits = dnorm_logits / dlogits * dL / dnorm_logits
dlogits = dnorm_logits.clone()

# logit_maxes = logits.max(1, keepdim=True).values
# this gets the highest of the values inside a given row inside the logits tensor and also squashes down into a [32, 1] tensor
# dL / dlogits = dlogits / dlogit_maxes * dL / dlogit_maxes
# we broadcast additional gradients to the maxes; to do this we encode the location with F.one_hot
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)

# logits = h @ W2 + b2. This is a matrix multiplication (dot product) of h with W2, then adding b2
# matrix multiplication has higher precedence, and then addition
# thus we backpropagate through the addition first, then the matrix multiplication
# db2 is just a passthrough, but it has shape [27], not [32, 27], so we sum the gradients
db2 = dlogits.sum(0)
cmp('b2', db2, b2)
 
# gradient of h will have shape [32, 64] compared to logits [32, 27]
# each element of h is affected by an entire row of W2 in the forward pass
# so in the backward pass, we need to reverse this operation by multiplying dlogits by W2.t()
dh = dlogits @ W2.t()
cmp('h', dh, h)

# h.t() is first because matrix multiplication requires the inner dimensions to match
dW2 = h.t() @ dlogits
cmp('W2', dW2, W2)

# h = torch.tanh(hpreact) # hidden layer
# dL / dhpreact = dL / dh * dh / dhpreact
# reversing the tanh operation -> y = tanh(z). dy / dz = 1 - y^2
dhpreact = (1.0 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

# hpreact = bngain * bnraw + bnbias
# bngain is [1, 64]; bnraw is [32, 64]; bnbias is [1, 64]; hpreact is [32, 64]
# so bnraw is broadcasted to hpreact shape, replicating rows in bngain.
# we first backpropagate through the multiplication then through the replication.
dbngain = (bnraw * dhpreact).sum(0, keepdims=True)
cmp('bngain', dbngain, bngain)

dbnbias = dhpreact.sum(0, keepdims=True)
cmp('bnbias', dbnbias, bnbias)

dbnraw = bngain * dhpreact
cmp('bnraw', dbnraw, bnraw)

# bnraw = bndiff * bnvar_inv
dbnvar_inv = (bndiff*dbnraw).sum(0, keepdims=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
dbndiff = bnvar_inv * dbnraw

# bnvar_inv = (bnvar + 1e-5)**-0.5
# = (bnvar + 10^-5)^-0.5
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
# there is a sum operation here, so we need to backpropagate through that
# dcounts += torch.ones_like(counts) * dcounts_sum # multiplying each row (torch.ones just to get the right shape), each element in a given row gets same gradient
dbndiff2 = torch.ones_like(bndiff2) * ((n-1)**-1) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

# bndiff2 = bndiff**2
dbndiff += 2 * bndiff * dbndiff2 
cmp('bndiff', dbndiff, bndiff) # didn't call this until later because dbndiff not finished proprogating yet

# bndiff = hprebn - bnmeani
# we need to sum the gradients for the rows 
dbnmeani = -dbndiff.sum(0, keepdims=True)
cmp('bnmeani', dbnmeani, bnmeani)
dhprebn = dbndiff.clone()

# bnmeani = (1/n)*hprebn.sum(0, keepdim=True)
# this is a sum and then a multiple, so backprop multiply first then spread into all elements
# sum already happened with the dbndiff clone -- correct dimensions
dhprebn += (1/n) * dbnmeani
cmp('hprebn', dhprebn, hprebn)

# hprebn = embcat @ W1 + b1
dembcat = dhprebn @ W1.t()
cmp('embcat', dembcat, embcat)

dW1 = embcat.t() @ dhprebn
cmp('W1', dW1, W1) 

db1 = dhprebn.sum(0)
cmp('b1', db1, b1)

# embcat = emb.view(emb.shape[0], -1)
# this basically reshapes the emb tensor into a 2D tensor, combining the y and z dimensions into one
# instead, reverse it and proprogate the dembcat straight through, but in the updated shape
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

# emb = emb_lookup[Xb]
# Xb is the indices of the characters in the minibatch for which embeddings are needed
# emb_lookup is the embedding matrix
# indices in Xb are used to select the corresponding rows from emb_lookup. For each index in Xb, the corresponding row (embedding vector) from emb_lookup is selected.
# Forward pass operations: 1) for each index in Xb, select corresponding row from emb_lookup. 2) concetenate the rows to form a 2D tensor.
# Each row in emb_lookup is used multiple times -> sum of the gradients of all outputs it influences. This summing reflects the total contribution of that vector to the loss across all its occurrences in the batch
# So, I should count how many of each index in Xb, multiply corresponding gradients
# I don't need to actually think about what index is in Xb - the gradients are already computed and flowing back - I just need to attach the gradients calaculated to each element in Xb, and add

demb_lookup = torch.zeros_like(emb_lookup) # to store gradients for each vocab char in emb_lookup
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]): # iterating through every character of Xb
        demb_lookup[Xb[i, j]] += demb[i, j]

cmp('emb_lookup', demb_lookup, emb_lookup)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
# TODO: better understand when to sum and when to average gradients.
# I start getting confused when I have a couple vectors of very different shapes to transport gradients between